# Harmonization across studies with PIC-SURE

This tutorial notebook will demonstrate how to query and work with the BioData Catalyst studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

 -------   

# Environment set-up

### System requirements
- Python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed


### Install packages

In [ ]:
import pandas as pd
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git@new-search

import PicSureBdcAdapter

## Connecting to a PIC-SURE network

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

 -------   

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate how to find and extract similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the variable *"orthopnea"* across studies with age


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the [`2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb`](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/blob/master/NHLBI_BioData_Catalyst/python/2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb) notebook*

-----

### Sex and gender variables across studies
<font color='darkgreen'>**Goal: Create a single harmonized variable called "sex" which combines data from multiple studies**</font> 

These variables are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

Let's start by searching for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms.



In [ ]:
sex_variable_dictionary = bdc.useDictionary().dictionary().find('sex|gender').dataframe()
print(sex_variable_dictionary.shape)
sex_variable_dictionary.head()


After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- TOPMed Harmonized data set (phs####)
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) (phs001252)
- EOCOPD (Early Onset of COPD) (phs000946)

We can find the study IDs in the data access dashboard in the user interface.

We have already used the PIC-SURE API dictionary object and find method to find variables which contain the keywords sex and gender. Let's filter these results to our desired studies of interest.

In [ ]:
list(sex_variable_dictionary)

In [ ]:
# which sex/gender variables are part of ECLIPSE?
eclipse_vars = sex_variable_dictionary['studyId'].str.contains('phs001252')

# create a subset of only ECLIPSE sex/gender vars
eclipse_sex_dictionary = sex_variable_dictionary[eclipse_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
eclipse_sex_dictionary


In [ ]:
# which sex/gender variables are part of EOCOPD?
eocopd_vars = sex_variable_dictionary['studyId'].str.contains('phs000946')

# create a subset of only EOCOPD sex/gender vars
eocopd_sex_dictionary = sex_variable_dictionary[eclipse_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
eocopd_sex_dictionary


We can see that there are many sex/gender variables in ECLIPSE originating from various data tables, and only one found in EOCOPD/

In [ ]:
# Initialize a query
authPicSure = bdc.useAuthPicSure()
sex_query = authPicSure.query()


In [ ]:
# Build query using selected concept paths
sex_query.anyof().add(pd.concat([eclipse_sex_dictionary['HPDS_PATH'], eocopd_sex_dictionary['HPDS_PATH']]))


In [ ]:
sex_results = sex_query.getResultsDataFrame()
sex_results.head()

# in progress - how to know which to pick?
Since there are multiple concept paths that are contain either `gender` or `sex` in the EOCOPD dataset, we can investigate these concept paths to determine the true variable of interest.

In [ ]:
# Uncomment following line to see full list of sex/gender variables in EOCOPD
#print("Full list of variables", eocopd_vars)

# Based on this, we can see that the variable we want for this analysis is the last in the list: Gender of participant
eocopd_var = find_vars(eocopd, ['gender of participant'])
print("Concept path from EOCOPD data set:\n", eocopd_var)

As part of our research, let's also say we are interested in body mass index (BMI) measurements across these studies. Let's save these concept paths to use in our queries as well.

In [ ]:
# search for body mass index variables
bmi_variable_dictionary = bdc.useDictionary().dictionary().find('body mass index').dataframe()

# which sex/gender variables are part of ECLIPSE or EOCOPD?
bmi_vars = bmi_variable_dictionary['studyId'].str.contains('phs001252|phs000946')

# create a subset of only the ECLIPSE / EOCOPD BMI vars
bmi_dictionary = bmi_variable_dictionary[bmi_vars][['studyId', 'dataTableName', 'name', 'description', 'values', 'HPDS_PATH']]
bmi_dictionary

In [ ]:
bmi_query = authPicSure.query()

# Build query using selected concept paths
bmi_query.anyof().add(bmi_dictionary['HPDS_PATH'])

# Get query results
bmi_results = bmi_query.getResultsDataFrame()
bmi_results.head()

Now that we know and have saved our concept paths of interest, we can use these to build our query. 

**Note: queries with the TOPMed DCC Harmonized data set cannot be combined with concept paths from other datasets. Because of this, we will run separate queries and combine the dataframes.**

Now that we have our patient-level dataframes, we can combine them into a single, cohesive dataframe.

The following function accomplishes three main tasks:
1. Removes extra columns, such as consent information
2. Renames the BMI and Sex columns
3. Adds the Dataset column, which corresponds to the study

In [ ]:
def clean_up_df(df, study):
    columns_to_drop = ['\\_Parent Study Accession with Subject ID\\', '\\_Topmed Study Accession with Subject ID\\', '\\_consents\\', '\\_harmonized_consent\\']
    df1 = df.drop(columns=columns_to_drop, errors='ignore')
    if 'body mass index' in df1.columns.values[1].lower():
        df1.columns = ['Patient ID', 'BMI', 'Sex']
    else:
        df1.columns = ['Patient ID', 'Sex', 'BMI']
    df2 = df1.dropna(subset=['BMI'])
    df2['Dataset'] = study
    return df2

In [ ]:
clean_eclipse = clean_up_df(eclipse_results, 'ECLIPSE')
clean_eocopd = clean_up_df(eocopd_results, 'EOCOPD')
clean_dcc = clean_up_df(dcc_harmonized_results, 'TOPMed Harmonized')

The datasets have been prepped. We can now merge them and begin our analysis.

In [ ]:
# Combine individual dataframes
final_df = pd.concat([clean for clean in [clean_eclipse, clean_eocopd, clean_dcc]], ignore_index=True)

In [ ]:
# Comparison of the datasets and sample harmonization
separate = final_df.drop(columns = ['Patient ID']).groupby(['Dataset','Sex']).mean()
print(separate)
harmonized = final_df.drop(columns = ['Patient ID']).groupby(['Sex']).mean()
print(harmonized)

In [ ]:
# Make lists of male and female mean BMI for plotting
male_means = list(separate[separate.index.get_level_values('Sex')=='Male']['BMI'])
male_means.append(*list(harmonized[harmonized.index.get_level_values('Sex')=='Male']['BMI']))
female_means = list(separate[separate.index.get_level_values('Sex')=='Female']['BMI'])
female_means.append(*list(harmonized[harmonized.index.get_level_values('Sex')=='Female']['BMI']))

In [ ]:
# Bar plot of the results
width = 0.2
labels = ['Male', 'Female']
x = np.arange(len(labels))

fig, ax = plt.subplots()
study1 = ax.bar(x - width*1.5, [male_means[0], female_means[0]], width, label='ECLIPSE')
study2 = ax.bar(x - width*0.5, [male_means[1], female_means[1]], width, label='EOCOPD')
study3 = ax.bar(x + width*0.5, [male_means[2], female_means[2]], width, label='TOPMed Harmonized')
study4 = ax.bar(x + width*1.5, [male_means[3], female_means[3]], width, label='Combined')

ax.set_ylabel('Body Mass Index (BMI)')
ax.set_title('Average Body Mass Index by Sex and Dataset')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.tight_layout()

plt.show()

-----

### Orthopnea variables across studies
In this example, we will be harmonizing variables related to `orthopnea`. Orthopnea is shortness of breath that occurs when individuals lie flat. Because of this, people with orthopnea have to sleep propped up in bed or sitting in a chair. You can read more about this condition [here](https://www.sleepfoundation.org/sleep-apnea/orthopnea).

Let's start by doing a search in PIC-SURE for concept paths containing `orthopnea`.

In [ ]:
ortho_dict = resource.dictionary().find("orthopnea").DataFrame()
ortho_dict.head()

As shown in the resulting dataframe, orthopnea is often recorded as a something like `sleep on two or more pillows to help you breathe?`

Let's do a search for `pillows` in PIC-SURE to explore those concept paths.

In [ ]:
pillow_dict = resource.dictionary().find("pillows").DataFrame()
pillow_dict.tail()

As shown in the resulting dataframe, there are some concept paths related to orthopnea that were not captured by the previous `orthopnea` search, such as `Do you sleep on 2 or more pillows to improve your breathing?` from the Jackson Heart Study (JHS) Cohort (phs000286). In fact, the JHS dataset was not returned at all when we searched for `orthopnea`. 

This is problematic for researchers studying these types of variables, since the concept paths of interest may not always be captured by a single search term. To account for this, researchers must conduct exploratory searches to determine potential search terms.

Let's say we want to harmonize orthopnea variables from the following datasets: FHS, MESA, and WHI. We can first get all concept paths in each study and then search for our terms of interest (`orthopnea` or `pillows`) to retrieve the concept paths of interest.

In [ ]:
# Function that filters out variables from a dataframe (df) that contain any of the terms (list_of_terms)
def find_concept_paths(study_name, list_of_terms):
    df = resource.dictionary().find(study_name).DataFrame()
    regex_version = '('+('|').join(list_of_terms)+')'
    var_filter = df.index.str.contains(regex_version, flags=re.IGNORECASE)
    vars_list = list(df[var_filter].index)
    return vars_list

In [ ]:
ortho_terms = ['orthopnea', 'pillows']

In [ ]:
mesa = find_concept_paths('Multi-Ethnic Study of Atherosclerosis (MESA) SHARe ( phs000209 )', ortho_terms)
# mesa # Uncomment to view full results list
mesa_var = mesa[0]
mesa_var

In [ ]:
whi = find_concept_paths("Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )", ortho_terms)
# whi # Uncomment to view full results list
whi_var = whi[0]
whi_var

In [ ]:
fhs = find_concept_paths("Framingham Cohort ( phs000007 )", ortho_terms)
fhs # Uncomment to view full results list
fhs_var = fhs[37]
fhs_var

As part of our research, let's say we are interested in exploring the relationship between pneumonia and orthopnea. Let's save concept paths related to `pneumonia` as well. 

In [ ]:
mesa_pneu = find_concept_paths('Multi-Ethnic Study of Atherosclerosis (MESA) SHARe ( phs000209 )', ['pneumonia'])
#mesa_pneu # Uncomment to view full results list
mesa_pneu_var = mesa_pneu[4]
mesa_pneu_var

In [ ]:
whi_pneu = find_concept_paths("Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )", ['pneumonia'])
# whi_pneu # Uncomment to view full results list
whi_pneu_var = whi_pneu[0]
whi_pneu_var

In [ ]:
fhs_pneu = find_concept_paths("Framingham Cohort ( phs000007 )", ['pneumonia'])
# fhs_pneu # Uncomment to view full results
fhs_pneu_var = fhs_pneu[26]
fhs_pneu_var

Now that we know and have saved our concept paths of interest, we can use these to build our queries and obtain our dataframes. 

In [ ]:
mesa_query = resource.query()
mesa_query.anyof().add(mesa_var)
mesa_query.require().add(mesa_pneu_var)
mesa_results = mesa_query.getResultsDataFrame(low_memory=False)

In [ ]:
fhs_query = resource.query()
fhs_query.anyof().add(fhs_var)
fhs_query.require().add(fhs_pneu_var)
fhs_results = fhs_query.getResultsDataFrame(low_memory=False)

In [ ]:
whi_query = resource.query()
whi_query.anyof().add(whi_var)
whi_query.require().add(whi_pneu_var)
whi_results = whi_query.getResultsDataFrame(low_memory=False)

Now that we have our patient-level dataframes, we can combine them into a single, cohesive dataframe.

The following function accomplishes three main tasks:
1. Removes extra columns, such as consent information
2. Renames the Pneumonia and Orthopnea columns
3. Adds the Dataset column, which corresponds to the study

In [ ]:
def clean_up_df2(df, study):
    columns_to_drop = ['\\_Parent Study Accession with Subject ID\\', '\\_Topmed Study Accession with Subject ID\\', '\\_consents\\', '\\_harmonized_consent\\']
    df1 = df.drop(columns=columns_to_drop, errors='ignore')
    if 'pneumonia' in df1.columns.values[1].lower():
        df1.columns = ['Patient ID', 'Pneumonia', 'Orthopnea']
    else:
        df1.columns = ['Patient ID', 'Orthopnea', 'Pneumonia']
    df1['Dataset'] = study
    return df1

In [ ]:
clean_fhs = clean_up_df2(fhs_results, 'FHS')
clean_whi = clean_up_df2(whi_results, 'WHI')
clean_mesa = clean_up_df2(mesa_results, 'MESA')

As you may have noticed, the orthopnea and pneumonia variables are encoded differently between these studies. Specifically:
- In FHS, pneumonia is recorded as "NO", "MAYBE", or "YES"
- In MESA, pneumonia is recorded as "Yes", "No", or "Don't know / Not sure"
- In WHI, pneumonia is recoded as a value, or the number of times a participant has been diagnosed with pneumonia

To harmonize these variables, we need to create a consistent encoding across these studies. 

In [ ]:
# Combine the results for the pneumonia and orthopnea variables
pneumonia_results = list(clean_fhs['Pneumonia'].unique())+list(clean_whi['Pneumonia'].unique())+list(clean_mesa['Pneumonia'].unique())
orthopnea_results = list(clean_fhs['Orthopnea'].unique())+list(clean_whi['Orthopnea'].unique())+list(clean_mesa['Orthopnea'].unique())

#pneumonia_results # Uncomment to view values
#orthopnea_results # Uncomment to view values

The following code creates these mappings:

|Raw value|Harmonized value|
|---|---|
|No|No|
|NO|No|
|No/Not recorded|No|
|Numeric value equal to 0|No|
|MAYBE|Maybe|
|Don't know / Not sure|Do not know|
|DO NOT KNOW|Do not know|
|YES|Yes|
|Yes|Yes|
|Numeric value greater than 0|Yes|

In [ ]:
mappings = {}
for res in [pneumonia_results, orthopnea_results]:
    for val in res:
        if val not in mappings.keys():
            if isinstance(val, float):
                if val == 0.0:
                    mappings[val] = 'No'
                else:
                    mappings[val] = 'Yes'
            else:
                if val.lower() == 'yes':
                    mappings[val] = 'Yes'
                if 'know' in val.lower():
                    mappings[val] = 'Do not know'
                elif 'no' in val.lower():
                    mappings[val] = 'No'
                if 'maybe' in val.lower():
                    mappings[val] = 'Maybe'

final_df = pd.concat([clean for clean in [clean_fhs, clean_whi, clean_mesa]], ignore_index=True)
final_df = final_df.replace({"Pneumonia": mappings, 'Orthopnea':mappings})
final_df.head()

Now that the variables have been encoded in the same way, we can use this dataframe for analysis and visualizations. 

In [ ]:
# library
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
# Filters to apply to the final_df to create figures
pneu_yes = final_df['Pneumonia'] == 'Yes'
pneu_no = final_df['Pneumonia'] == 'No'
ortho_yes = final_df['Orthopnea'] == 'Yes'
ortho_no = final_df['Orthopnea'] == 'No'
fhs = final_df['Dataset'] == 'FHS'
mesa = final_df['Dataset'] == 'MESA'
whi = final_df['Dataset'] == 'WHI'

In [ ]:
# FHS figure
pneu_count =  final_df[pneu_yes & ortho_no & fhs].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & fhs].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & fhs].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("FHS Dataset")
plt.show()

# MESA figure
pneu_count =  final_df[pneu_yes & ortho_no & mesa].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & mesa].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & mesa].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("MESA Dataset")
plt.show()

# WHI figure
pneu_count =  final_df[pneu_yes & ortho_no & whi].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes & whi].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes & whi].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("WHI Dataset")
plt.show()

# Combined figure
pneu_count =  final_df[pneu_yes & ortho_no].count()['Patient ID']
ortho_count = final_df[pneu_no & ortho_yes].count()['Patient ID']
both_count =  final_df[pneu_yes & ortho_yes].count()['Patient ID']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("Harmonized")
plt.show()